In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mock-competition-2-ai-funaab/sample_submission.csv
/kaggle/input/mock-competition-2-ai-funaab/test.csv
/kaggle/input/mock-competition-2-ai-funaab/train.csv
/kaggle/input/mock-competition-2-ai-funaab/VariableDescription.csv


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [3]:
train =pd.read_csv('/kaggle/input/mock-competition-2-ai-funaab/train.csv',parse_dates=['YearOfObservation'],)
train.set_index('Customer Id', inplace=True)
train.head()

,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
Customer Id,,,,,,,,,,,,,
H3793,2013-01-01,1.000000,0,V,N,O,R,900.0,1,1981.0,3,15014,1
H18227,2012-01-01,0.915301,0,V,N,O,R,1672.0,1,1983.0,4,36044,1
H13985,2016-01-01,1.000000,0,N,V,V,U,420.0,2,1988.0,.,13001,0
H17669,2013-01-01,1.000000,0,V,N,O,R,1956.0,2,1960.0,4,83009,1
H4156,2012-01-01,1.000000,0,V,V,V,U,800.0,1,1960.0,.,91174,0


In [4]:
test =pd.read_csv('/kaggle/input/mock-competition-2-ai-funaab/test.csv',parse_dates=['YearOfObservation'],)

test.set_index('Customer Id', inplace=True)
test.head()

,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code
Customer Id,,,,,,,,,,,,
H18966,2012-01-01,0.259563,0,V,N,O,R,1425.0,2,1950.0,7,13205
H16694,2016-01-01,0.997268,0,V,N,O,R,1264.0,3,1980.0,5,93048
H18661,2013-01-01,1.000000,0,V,N,O,R,2600.0,2,1962.0,4,30133
H18872,2013-01-01,1.000000,1,V,N,O,R,1015.0,3,1920.0,5,92012
H18032,2014-01-01,1.000000,1,V,N,O,R,15620.0,3,1978.0,>=10,59350


In [5]:
sample =pd.read_csv('/kaggle/input/mock-competition-2-ai-funaab/sample_submission.csv')
sample.head(2)

,Customer Id,Claim
0,H18966,1
1,H16694,0


In [6]:
obj_df = train.select_dtypes(include=['object'])
obj_df.head()

,Building_Painted,Building_Fenced,Garden,Settlement,NumberOfWindows,Geo_Code
Customer Id,,,,,,
H3793,V,N,O,R,3,15014
H18227,V,N,O,R,4,36044
H13985,N,V,V,U,.,13001
H17669,V,N,O,R,4,83009
H4156,V,V,V,U,.,91174


In [7]:
cat_df = test.select_dtypes(include=['object'])

In [8]:
obj_df[obj_df.isnull().any(axis=1)]

,Building_Painted,Building_Fenced,Garden,Settlement,NumberOfWindows,Geo_Code
Customer Id,,,,,,
H16605,V,V,V,U,.,NaN
H15115,V,V,V,U,.,NaN
H19266,V,V,V,U,.,NaN
H18878,V,V,V,U,.,NaN
H3915,V,N,NaN,R,4,73015
...,...,...,...,...,...,...
H15876,V,V,V,U,.,NaN
H16496,V,V,V,U,.,NaN
H4372,V,V,V,U,.,NaN


In [9]:
cat_df[cat_df.isnull().any(axis=1)]

,Building_Painted,Building_Fenced,Garden,Settlement,NumberOfWindows,Geo_Code
Customer Id,,,,,,
H17510,V,V,V,U,.,NaN
H6204,V,V,V,U,.,NaN
H3238,V,V,V,U,.,NaN
H19156,V,V,V,U,.,NaN
H1571,V,N,NaN,R,5,74173
H19055,V,V,V,U,.,NaN
H13531,V,V,V,U,.,NaN
H17241,V,V,V,U,.,NaN
H16606,V,V,V,U,.,NaN


In [10]:
cat_df["NumberOfWindows"].value_counts()

   .    1053
4        292
3        254
5        194
2        123
6         90
7         52
8         34
1         22
>=10      21
9         13
Name: NumberOfWindows, dtype: int64

In [11]:
cat_df["Geo_Code"].value_counts()

6083     43
33063    42
6088     38
31555    31
92004    29
         ..
38140     1
95060     1
40209     1
90104     1
67169     1
Name: Geo_Code, Length: 710, dtype: int64

In [12]:
obj_df["NumberOfWindows"].value_counts()

   .    2498
4        647
3        590
5        445
2        240
6        216
7        159
8         82
1         53
>=10      46
9         36
Name: NumberOfWindows, dtype: int64

In [13]:
train.dtypes

YearOfObservation     datetime64[ns]
Insured_Period               float64
Residential                    int64
Building_Painted              object
Building_Fenced               object
Garden                        object
Settlement                    object
Building Dimension           float64
Building_Type                  int64
Date_of_Occupancy            float64
NumberOfWindows               object
Geo_Code                      object
Claim                          int64
dtype: object

In [14]:
list(set(obj_df.Geo_Code.unique()))

[nan,
 '17461',
 '30007',
 '59482',
 '64237',
 '42005',
 '75118',
 '30351',
 '62571',
 '59025',
 '34023',
 '13096',
 '94069',
 '34037',
 '22113',
 '73006',
 '13031',
 '74293',
 '77053',
 '74016',
 '63284',
 '78481',
 '64129',
 '68271',
 '74133',
 '78513',
 '74189',
 '74264',
 '77487',
 '74099',
 '66124',
 '88516',
 '81004',
 '6073',
 '91469',
 '92050',
 '88075',
 '2B042',
 '30341',
 '9301',
 '83036',
 '54482',
 '25388',
 '64122',
 '31446',
 '64024',
 '38151',
 '69243',
 '33522',
 '42110',
 '16292',
 '6069',
 '57663',
 '74224',
 '68161',
 '95598',
 '73176',
 '68126',
 '91386',
 '67948',
 '67218',
 '91215',
 '60346',
 '68297',
 '38138',
 '69052',
 '68013',
 '82112',
 '33366',
 '64189',
 '30133',
 '61001',
 '90008',
 '91223',
 '73179',
 '42183',
 '42222',
 '14117',
 '94068',
 '77152',
 '54321',
 '93046',
 '40209',
 '6036',
 '64160',
 '2A004',
 '84129',
 '13063',
 '6128',
 '92073',
 '38185',
 '13202',
 '76231',
 '69297',
 '91027',
 '30003',
 '47069',
 '45234',
 '34113',
 '90075',
 '35284',

'2A090''2A004''2A247''2B033'

In [45]:
obj_df = obj_df.fillna({"Garden": 0})

In [46]:
cat_df = cat_df.fillna({"Garden": 0})

In [47]:
obj_df["Garden"].unique()

array(['O', 'V'], dtype=object)

In [48]:
obj["Garden"].value_counts()

0    2514
1    2498
Name: Garden, dtype: int64

In [19]:
obj_df["Geo_Code"].value_counts()

6088     105
33063     95
13206     76
6083      70
63113     63
        ... 
95203      1
38006      1
56181      1
22094      1
17461      1
Name: Geo_Code, Length: 1127, dtype: int64

In [20]:
obj_df["Geo_Code"].value_counts()

6088     105
33063     95
13206     76
6083      70
63113     63
        ... 
95203      1
38006      1
56181      1
22094      1
17461      1
Name: Geo_Code, Length: 1127, dtype: int64

In [21]:
obj_df["Geo_Code"].value_counts()

6088     105
33063     95
13206     76
6083      70
63113     63
        ... 
95203      1
38006      1
56181      1
22094      1
17461      1
Name: Geo_Code, Length: 1127, dtype: int64

In [22]:
obj_df = obj_df.fillna({"Geo_Code": "6088"})

In [23]:
cat_df = cat_df.fillna({"Geo_Code": "6088"})

In [25]:
obj=obj_df.drop(['Geo_Code'],axis=1)

In [26]:
cat=cat_df.drop(['Geo_Code'],axis=1)

In [27]:
nums = {
                "Building_Painted": {"V": 1,"N": 0},
                 'Building_Fenced':{'V':1,'N':0},
                   'Garden':{'V':1,'O':0,'<':0},
               'Settlement':{'U':1,'R':0 },
               'NumberOfWindows':{'   .': -999,">=10":10}}

In [28]:
obj.replace(nums, inplace=True)
obj.head()

,Building_Painted,Building_Fenced,Garden,Settlement,NumberOfWindows
Customer Id,,,,,
H3793,1,0,0,0,3
H18227,1,0,0,0,4
H13985,0,1,1,1,-999
H17669,1,0,0,0,4
H4156,1,1,1,1,-999


In [29]:
cat.replace(nums, inplace=True)
cat.head()

,Building_Painted,Building_Fenced,Garden,Settlement,NumberOfWindows
Customer Id,,,,,
H18966,1,0,0,0,7
H16694,1,0,0,0,5
H18661,1,0,0,0,4
H18872,1,0,0,0,5
H18032,1,0,0,0,10


In [30]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train['Building_Painted']= le.fit_transform(train['Building_Painted'])
train['Building_Fenced']= le.fit_transform(train['Building_Fenced'])
train['Settlement']= le.fit_transform(train['Settlement'])
train['NumberOfWindows']=le.fit_transform(train['NumberOfWindows'])



In [33]:
test['Building_Painted']= le.fit_transform(test['Building_Painted'])
test['Building_Fenced']= le.fit_transform(test['Building_Fenced'])
test['Settlement']= le.fit_transform(test['Settlement'])
test['NumberOfWindows']=le.fit_transform(test['NumberOfWindows'])


In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5012 entries, H3793 to H1576
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YearOfObservation   5012 non-null   int64  
 1   Insured_Period      5012 non-null   float64
 2   Residential         5012 non-null   int64  
 3   Building_Painted    5012 non-null   int64  
 4   Building_Fenced     5012 non-null   int64  
 5   Garden              5008 non-null   object 
 6   Settlement          5012 non-null   int64  
 7   Building Dimension  5012 non-null   float64
 8   Building_Type       5012 non-null   int64  
 9   Date_of_Occupancy   5012 non-null   int64  
 10  NumberOfWindows     5012 non-null   int64  
 11  Geo_Code            4943 non-null   object 
 12  Claim               5012 non-null   int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 548.2+ KB


In [31]:
obj.dtypes

Building_Painted     int64
Building_Fenced      int64
Garden               int64
Settlement           int64
NumberOfWindows     object
dtype: object

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5012 entries, H3793 to H1576
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YearOfObservation   5012 non-null   int64  
 1   Insured_Period      5012 non-null   float64
 2   Residential         5012 non-null   int64  
 3   Building_Painted    5012 non-null   int64  
 4   Building_Fenced     5012 non-null   int64  
 5   Garden              5008 non-null   object 
 6   Settlement          5012 non-null   int64  
 7   Building Dimension  4940 non-null   float64
 8   Building_Type       5012 non-null   int64  
 9   Date_of_Occupancy   4661 non-null   float64
 10  NumberOfWindows     5012 non-null   int64  
 11  Geo_Code            4943 non-null   object 
 12  Claim               5012 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 548.2+ KB


In [36]:
train['Building Dimension'] = train['Building Dimension'].fillna(-999)
train.Date_of_Occupancy = train.Date_of_Occupancy.fillna(1960)
train.Date_of_Occupancy = train.Date_of_Occupancy.astype('int64')
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5012 entries, H3793 to H1576
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YearOfObservation   5012 non-null   int64  
 1   Insured_Period      5012 non-null   float64
 2   Residential         5012 non-null   int64  
 3   Building_Painted    5012 non-null   int64  
 4   Building_Fenced     5012 non-null   int64  
 5   Garden              5008 non-null   object 
 6   Settlement          5012 non-null   int64  
 7   Building Dimension  5012 non-null   float64
 8   Building_Type       5012 non-null   int64  
 9   Date_of_Occupancy   5012 non-null   int64  
 10  NumberOfWindows     5012 non-null   int64  
 11  Geo_Code            4943 non-null   object 
 12  Claim               5012 non-null   int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 548.2+ KB


In [37]:
train.isnull().sum()* 100/ len(train)


YearOfObservation     0.000000
Insured_Period        0.000000
Residential           0.000000
Building_Painted      0.000000
Building_Fenced       0.000000
Garden                0.079808
Settlement            0.000000
Building Dimension    0.000000
Building_Type         0.000000
Date_of_Occupancy     0.000000
NumberOfWindows       0.000000
Geo_Code              1.376696
Claim                 0.000000
dtype: float64

In [31]:
train['YearOfObservation'] = train.YearOfObservation.astype(int)

In [32]:
test['YearOfObservation'] = test.YearOfObservation.astype(int)

In [38]:
test['Building Dimension'] = test['Building Dimension'].fillna(-999)
test.Date_of_Occupancy = test.Date_of_Occupancy.fillna(1960)
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2148 entries, H18966 to H3759
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YearOfObservation   2148 non-null   int64  
 1   Insured_Period      2148 non-null   float64
 2   Residential         2148 non-null   int64  
 3   Building_Painted    2148 non-null   int64  
 4   Building_Fenced     2148 non-null   int64  
 5   Garden              2145 non-null   object 
 6   Settlement          2148 non-null   int64  
 7   Building Dimension  2148 non-null   float64
 8   Building_Type       2148 non-null   int64  
 9   Date_of_Occupancy   2148 non-null   float64
 10  NumberOfWindows     2148 non-null   int64  
 11  Geo_Code            2115 non-null   object 
dtypes: float64(3), int64(7), object(2)
memory usage: 218.2+ KB


In [37]:
test['Garden']=test.Garden.astype(str).astype(int)

ValueError: invalid literal for int() with base 10: 'O'

In [36]:
test['Garden']= le.fit_transform(test['Garden'])
train['Garden']= le.fit_transform(train['Garden'])

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']

In [39]:
X = train.drop(['Garden','Claim','Geo_Code'], axis=1)
y = train.Claim
test_df= test.drop(['Garden','Geo_Code'],axis=1)

In [ ]:
sklearn.linear_model.LogisticRegression
lr = LogisticRegression(C = 1,max_iter = 3000,class_weight = 'balanced',random_state=123)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
confusion_matrix(y_test,y_pred)

In [61]:
from rgf.sklearn import RGFClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
clf = RandomForestClassifier(n_jobs=-1,random_state = 123)
cbt = CatBoostClassifier(verbose=False, thread_count=-1,random_state = 123)
rgf =RGFClassifier(verbose=False)
scale= MinMaxScaler()

model_classifiers= ('rgf',RGFClassifier( algorithm ='RGF_Sib',max_leaf =1000)),

In [66]:
model_classifiers=(('cbt',CatBoostClassifier(verbose=False, thread_count=-1,random_state = 123)),
                   ('rgf',RGFClassifier( algorithm ='RGF_Sib',max_leaf =1000)))

In [62]:
def model_score(model):
    train_roc = roc_auc_score(ytrain, model.predict(xtrain))
    test_roc = roc_auc_score(ytest, model.predict(xtest))
    print('Training data ROU_AUC score: {} \t Validation ROU_AUC_score: {}'.format(train_roc,test_roc))
    return test_roc

In [67]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=419)
scores,preds = [],[]
model= VotingClassifier(estimators= model_classifiers, voting='hard')
#the ensembled model is used to carry out the prediction probability with weights assignedto each algorithm to imcrease accuracy
j = 1
for train_split, test_split in skf.split(X, y):#Creating a cross validation for loop using StratifiedKFold
    pipe = Pipeline([('preprocessor', scale), ('model', model)])
    xtrain, xtest, ytrain, ytest = X.iloc[train_split], X.iloc[test_split], y.iloc[train_split], y.iloc[test_split]# splitting of the dataset for training and validation
    pipe.fit(xtrain, ytrain)
    print('Splits trained {}'.format(j))
    score = model_score(pipe)
    pred = pipe.predict(test)
    scores.append(score)
    preds.append(pred)
    j += 1
np.mean(scores)

TypeError: zip argument #2 must support iteration